In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.tsa.stattools import adfuller
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from scipy.stats import itemfreq




## Load datasets

In [2]:
no_attack=pd.read_csv('BATADAL_dataset03.csv')
attack=pd.read_csv('BATADAL_dataset04.csv')
#attack.drop(attack.index[4176], inplace=True)
test=pd.read_csv('BATADAL_test_dataset.csv')
attack.columns = attack.columns.str.strip()
attack['DATETIME']=pd.to_datetime(attack['DATETIME'])
no_attack['DATETIME']=pd.to_datetime(no_attack['DATETIME'])
test['DATETIME']=pd.to_datetime(test['DATETIME'])
attack.set_index('DATETIME', inplace=True)
no_attack.set_index('DATETIME', inplace=True)
test.set_index('DATETIME', inplace=True)
#used for final evaluations
dataset=pd.read_csv('BATADAL_dataset04.csv')
dataset.columns = dataset.columns.str.strip()


## Functions that are used in order to create the discrete models

In [3]:
def znormalization(ts):
    """
    ts - each column of ts is a time series (np.ndarray)
    """
    mus = ts.mean(axis = 0)
    stds = ts.std(axis = 0)
    return (ts - mus) / stds

def paa_transform(data, n_pieces):
    """
    ts: the columns of which are time series represented by e.g. np.array
    n_pieces: M equally sized piecies into which the original ts is splitted
    """
    splitted = np.array_split(data, n_pieces) ## along columns as we want
    return list(map(lambda xs: xs.mean(axis = 0), splitted))

def sax_transform(dat,alphabet):
    from scipy.stats import norm
    alphabet_sz = len(alphabet)
    data=dat
    thrholds = norm.ppf(np.linspace(1./alphabet_sz,1-1./alphabet_sz,alphabet_sz-1))
    for indx, i in enumerate(data):
        temp=min(enumerate(thrholds), key=lambda x: abs(x[1]-i))
        if i< temp[1]:
            data[indx]=alphabet[temp[0]]
        else:
            data[indx]=alphabet[temp[0]+1]
    return data

def n_grams(data):
    length=len(data)
    bigrams=[]
    for i in range(0,length):
        if i<length-2:
            bigram=data[i]+data[i+1]+data[i+2]
            bigrams.append(bigram)
    return bigrams

In [4]:
 signals= ['L_T1','L_T7','L_T4','L_T3','L_T2','F_PU3','F_PU6','F_PU7','F_PU3','F_PU11','F_PU1']
#if you want to test the results for all the signals (results are also presented in the report)
#you can just comment the previous signals and uncomment the folllowing variable:
# signals= ['L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7', 'F_PU1',
#           'S_PU1', 'F_PU2', 'F_PU3', 'F_PU4','F_PU5', 'F_PU6', 'F_PU7',
#           'F_PU8','F_PU9',  'F_PU10',  'F_PU11','S_PU11', 'F_V2', 'P_J280',
#           'P_J269', 'P_J300', 'P_J256','P_J289', 'P_J415', 'P_J302', 'P_J306', 'P_J307', 'P_J317','P_J14', 'P_J422']

## Creating the discrete models using as training set the first training dataset and as test the second training dataset

In [5]:
all_signals=[]
for sign in signals:
    print('#######################################')
    print('Processing sensor ',sign)
    print('#######################################')
    print('  ')
    signal_train=no_attack[sign]
    normalized_data=znormalization(signal_train)
    paa_form=paa_transform(normalized_data,116)
    sax_data=sax_transform(paa_form,'abcd')
    sax_data=''.join(sax_data)
    print('SAX format of training data')
    print(sax_data)
    print('  ')
    trigrams=n_grams(sax_data)
    print('SAX format of training data in tri-grams')
    print(trigrams)
    print('  ')
    trigrams_freq=itemfreq(trigrams)
    signal_test=attack[sign]
    normalized_data_test=znormalization(signal_test)
    paa_form_test=paa_transform(normalized_data_test,116)
    sax_data_test=sax_transform(paa_form_test,'abcd')
    sax_data_test=''.join(sax_data_test)
    print('SAX format of testing data')
    print(sax_data_test)
    print('  ')
    trigrams_test=n_grams(sax_data_test)
    print('SAX format of testing data in tri-grams')
    print(trigrams_test)
    print('  ')
    trigrams_freq_test=itemfreq(trigrams_test)
    anomalies=[]
    for i in trigrams_freq_test:
        if i[0] not in trigrams_freq:
            anomalies.append(i[0])
    tp=0
    fp=0
    for i in enumerate(trigrams_test):
        if i[1] in anomalies:
            signal_bottom=i[0]*36
            signal_top=signal_bottom+36
            all_signals.append(i[0])
            for j in range(signal_bottom,signal_top):
                if dataset.ATT_FLAG[j]==1:
                    tp+=1
                else:
                    fp+=1
    tn=dataset.loc[dataset.ATT_FLAG==-999].shape[0]-fp
    fn=dataset.loc[dataset.ATT_FLAG==1].shape[0]-tp
    Accuracy=100.0*(tp+tn)/(tp+tn+fp+fn)
    if (tp+fp)!=0:
        Precision=100.0*tp / (tp + fp)
    else:
         Precision=0
    Recall = 100.0*tp / (tp + fn)
    F_score = 100.0*2*tp /(2*tp + fp + fn)
    print ("TP:", tp)
    print ("FP:", fp)
    print("Accuracy: %.2f" % Accuracy)
    print("Precision: %.2f" % Precision)
    print("Recall: %.2f" %Recall)
    print("F_score: %.2f" % F_score)
    print('  ')

#######################################
('Processing sensor ', 'L_T1')
#######################################
  
SAX format of training data
bcbbccbbcccbccbcbbccbcbcbcccbcccbcbcbbcbbcbccbcbccbccbbcbccbccbbcbccbbcbcbcbbcbbcccbcbbbcbcbccbcccbbccccccbcbcbccbbc
  
SAX format of training data in tri-grams
['bcb', 'cbb', 'bbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cbb', 'bbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbb', 'bbc', 'bcb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccc', 'ccb', 'cbb', 'bbc', 'bc

('TP:', 73)
('FP:', 71)
Accuracy: 94.80
Precision: 50.69
Recall: 33.33
F_score: 40.22
  
#######################################
('Processing sensor ', 'F_PU3')
#######################################
  
SAX format of training data
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
  
SAX format of training data in tri-grams
['bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb

## Combine all individual discrete models to create a single discrete model for anomaly detection

In [6]:
tp=0
fp=0
final=[]
positions=[]
tot=set(all_signals)
for i in tot:
    if(all_signals.count(i)>1):
        final.append(i)
for i in final:
    signal_bottom=i*36
    signal_top=signal_bottom+36
    positions.append([signal_bottom,signal_top])
    for j in range(signal_bottom,signal_top): 
        if dataset.ATT_FLAG[j]==1:
            tp+=1
        else:
            fp+=1
tn=dataset.loc[dataset.ATT_FLAG==-999].shape[0]-fp
fn=dataset.loc[dataset.ATT_FLAG==1].shape[0]-tp
Accuracy=100.0*(tp+tn)/(tp+tn+fp+fn)
if (tp+fp)!=0:
    Precision=100.0*tp / (tp + fp)
else:
    Precision=0
Recall = 100.0*tp / (tp + fn)
F_score = 100.0*2*tp /(2*tp + fp + fn)
print ("TP:", tp)
print ("FP:", fp)
print("Accuracy: %.2f" % Accuracy)
print("Precision: %.2f" % Precision)
print("Recall: %.2f" %Recall)
print("F_score: %.2f" % F_score)

#when we run the system for all signals, we get the following results
# ('TP:', 203)
# ('FP:', 1021)
# Accuracy: 75.17
# Precision: 16.58
# Recall: 92.69
# F_score: 28.14



('TP:', 112)
('FP:', 320)
Accuracy: 89.78
Precision: 25.93
Recall: 51.14
F_score: 34.41


## Saving discrete model prediction in order to be used in comparison task

In [7]:
predicted_labels = np.ones((attack.shape[0]))*-999
for i in positions:
    predicted_labels[i[0]:i[1]]=1

predicted_labels=pd.DataFrame(predicted_labels)
predicted_labels.to_csv('sax_predictions.csv',index=False)

## Creating the discrete models using as training set the first training dataset and as test the test dataset

In [8]:
#Preprocessing in test data (adding labels based on the pdf that describe the attacks)

#set labels to test dataset based on the file of attacks
test['ATT_FLAG']=-999
test['ATT_FLAG']['2017-01-16 09:00:00':'2017-01-19 06:00:00']=1
test['ATT_FLAG']['2017-01-30 08:00:00':'2017-02-02 00:00:00']=1
test['ATT_FLAG']['2017-02-09 03:00:00':'2017-02-10 09:00:00']=1
test['ATT_FLAG']['2017-02-12 01:00:00':'2017-02-13 07:00:00']=1
test['ATT_FLAG']['2017-02-24 05:00:00':'2017-02-28 08:00:00']=1
test['ATT_FLAG']['2017-03-10 14:00:00':'2017-03-13 21:00:00']=1
test['ATT_FLAG']['2017-03-25 20:00:00':'2017-03-27 01:00:00']=1



In [9]:
all_signals=[]
for sign in signals:
    print('#######################################')
    print('Processing sensor ',sign)
    print('#######################################')
    print('  ')
    signal_train=no_attack[sign]
    normalized_data=znormalization(signal_train)
    paa_form=paa_transform(normalized_data,116)
    sax_data=sax_transform(paa_form,'abcd')
    sax_data=''.join(sax_data)
    print('SAX format of training data')
    print(sax_data)
    print('  ')
    trigrams=n_grams(sax_data)
    print('SAX format of training data in tri-grams')
    print(trigrams)
    print('  ')
    trigrams_freq=itemfreq(trigrams)
    signal_test=test[sign]
    normalized_data_test=znormalization(signal_test)
    paa_form_test=paa_transform(normalized_data_test,116)
    sax_data_test=sax_transform(paa_form_test,'abcd')
    sax_data_test=''.join(sax_data_test)
    print('SAX format of testing data')
    print(sax_data_test)
    print('  ')
    trigrams_test=n_grams(sax_data_test)
    print('SAX format of testing data in tri-grams')
    print(trigrams_test)
    print('  ')
    trigrams_freq_test=itemfreq(trigrams_test)
    anomalies=[]
    for i in trigrams_freq_test:
        if i[0] not in trigrams_freq:
            anomalies.append(i[0])
    tp=0
    fp=0
    for i in enumerate(trigrams_test):
        if i[1] in anomalies:
            signal_bottom=i[0]*18
            signal_top=signal_bottom+18
            all_signals.append(i[0])
            for j in range(signal_bottom,signal_top):
                if test.ATT_FLAG[j]==1:
                    tp+=1
                else:
                    fp+=1
    tn=test.loc[test.ATT_FLAG==-999].shape[0]-fp
    fn=test.loc[test.ATT_FLAG==1].shape[0]-tp
    Accuracy=100.0*(tp+tn)/(tp+tn+fp+fn)
    if (tp+fp)!=0:
        Precision=100.0*tp / (tp + fp)
    else:
         Precision=0
    Recall = 100.0*tp / (tp + fn)
    F_score = 100.0*2*tp /(2*tp + fp + fn)
    print ("TP:", tp)
    print ("FP:", fp)
    print("Accuracy: %.2f" % Accuracy)
    print("Precision: %.2f" % Precision)
    print("Recall: %.2f" %Recall)
    print("F_score: %.2f" % F_score)
    print('  ')

#######################################
('Processing sensor ', 'L_T1')
#######################################
  
SAX format of training data
bcbbccbbcccbccbcbbccbcbcbcccbcccbcbcbbcbbcbccbcbccbccbbcbccbccbbcbccbbcbcbcbbcbbcccbcbbbcbcbccbcccbbccccccbcbcbccbbc
  
SAX format of training data in tri-grams
['bcb', 'cbb', 'bbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cbb', 'bbc', 'bcc', 'ccc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbb', 'bbc', 'bcb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccc', 'ccb', 'cbb', 'bbc', 'bc

SAX format of training data
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbdbbbbbbcbcbbbbdbbbbbcbbbbbbcbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbcbbbbbbbbbbbbbbbbbbbb
  
SAX format of training data in tri-grams
['bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbd', 'bdb', 'dbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbc', 'bcb', 'cbc', 'bcb', 'cbb', 'bbb', 'bbb', 'bbd', 'bdb', 'dbb', 'bbb', 'bbb', 'bbb', 'bbc', 'bcb', 'cbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbc', 'bcb', 'cbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbc', 'bcb', 'cbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb', 'bbb']
  
SAX forma

SAX format of testing data
bbcbccbdbccbccbdbccbbdbcbbdbbdbbccbbdbbdbbdbccbcbbcbaccbbdbbccbdbbccbcbdbdbbccbcbbdbbdbbccbcbbccbbcbccbccbcbccbcdbbd
  
SAX format of testing data in tri-grams
['bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbd', 'bdb', 'dbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbd', 'bdb', 'dbc', 'bcc', 'ccb', 'cbb', 'bbd', 'bdb', 'dbc', 'bcb', 'cbb', 'bbd', 'bdb', 'dbb', 'bbd', 'bdb', 'dbb', 'bbc', 'bcc', 'ccb', 'cbb', 'bbd', 'bdb', 'dbb', 'bbd', 'bdb', 'dbb', 'bbd', 'bdb', 'dbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcb', 'cba', 'bac', 'acc', 'ccb', 'cbb', 'bbd', 'bdb', 'dbb', 'bbc', 'bcc', 'ccb', 'cbd', 'bdb', 'dbb', 'bbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbd', 'bdb', 'dbd', 'bdb', 'dbb', 'bbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbd', 'bdb', 'dbb', 'bbd', 'bdb', 'dbb', 'bbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbb', 'bbc', 'bcc', 'ccb', 'cbb', 'bbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcb', 'cbc', 'bcc', 'ccb', 'cbc', 'bcd', 'cdb', 'dbb', 'bbd']
  
('TP:', 153

## Combine all individual discrete models to create a single discrete model for anomaly detection

In [10]:
tp=0
fp=0
final=[]
positions=[]
tot=set(all_signals)
for i in tot:
    if(all_signals.count(i)>1):
        final.append(i)
for i in final:
    signal_bottom=i*18
    signal_top=signal_bottom+18
    positions.append([signal_bottom,signal_top])
    for j in range(signal_bottom,signal_top): 
        if test.ATT_FLAG[j]==1:
            tp+=1
        else:
            fp+=1
tn=test.loc[test.ATT_FLAG==-999].shape[0]-fp
fn=test.loc[test.ATT_FLAG==1].shape[0]-tp
Accuracy=100.0*(tp+tn)/(tp+tn+fp+fn)
if (tp+fp)!=0:
    Precision=100.0*tp / (tp + fp)
else:
    Precision=0
Recall = 100.0*tp / (tp + fn)
F_score = 100.0*2*tp /(2*tp + fp + fn)
print ("TP:", tp)
print ("FP:", fp)
print("Accuracy: %.2f" % Accuracy)
print("Precision: %.2f" % Precision)
print("Recall: %.2f" %Recall)
print("F_score: %.2f" % F_score)

#when we run the system for all signals, we get the following results
# ('TP:', 271)
# ('FP:', 1763)
# Accuracy: 15.61
# Precision: 13.32
# Recall: 100.00
# F_score: 23.51

('TP:', 210)
('FP:', 1158)
Accuracy: 41.65
Precision: 15.35
Recall: 77.49
F_score: 25.63
